In [46]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [47]:
from __future__ import division

import gym
import numpy as np
import pandas as pd
import random
import tensorflow as tf  

%load_ext tensorboard


import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow_addons.layers as layers


from gym import spaces

try:
    import cPickle as pickle
except:
    import pickle


import math

try:
    xrange = xrange
except:
    xrange = range

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [48]:
df_pca = pd.read_csv('./osi_pca_final.csv')

In [49]:
df_pca.head(10)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,Revenue
0,0.104711,0.115576,-0.395859,-0.367811,0.792326,0.687524,-0.413624,0.404863,-0.295999,-0.032112,-0.020025,0.141937,0.128971,-0.100672,-0.406829,0.845749,0
1,0.000601,0.142662,-0.271433,-0.129434,0.178537,-0.074198,-0.258909,0.315821,-0.120330,0.054817,0.010389,0.211168,0.171537,0.018455,-0.577434,0.774437,0
2,0.110375,0.146153,-0.392492,-0.241203,0.771216,0.936403,0.567604,0.373728,-0.208422,-0.012134,-0.036700,0.155926,0.128702,0.008547,-0.345934,0.866735,0
3,0.034761,0.133210,-0.310407,-0.177025,0.365475,0.202037,-0.171265,0.328262,-0.037567,0.074858,-0.003444,0.192169,0.157149,0.051173,-0.515762,0.798940,0
4,-0.123149,0.048001,0.711157,-0.095758,0.307547,-0.108562,-0.230177,0.309132,0.017062,0.116845,0.001683,0.205523,0.176376,0.135875,-0.541553,0.803390,0
5,-0.023538,0.142550,-0.244491,-0.076902,0.053693,-0.219202,-0.229715,0.302661,-0.056145,0.077405,0.014214,0.208672,0.174958,-0.000499,-0.546529,0.798287,0
6,0.183102,0.084289,-0.393380,-0.326629,0.790718,0.774777,-0.155713,0.354513,-0.149013,0.013143,-0.041774,0.523529,0.119604,0.115097,-0.333076,0.805666,0
7,-0.005150,0.012790,0.571897,-0.358637,0.986430,0.731866,-0.398297,0.403194,-0.096423,0.043948,-0.030007,0.131088,0.136232,-0.075089,-0.399655,0.872121,0
8,0.151072,0.067793,-0.260093,-0.120412,0.182449,-0.026001,-0.147373,0.247943,-0.057471,0.051713,-0.017834,0.966725,0.171442,-0.022431,-0.433824,0.671264,0
9,0.045417,0.112371,-0.232892,-0.063361,0.016315,-0.258814,-0.220094,0.264003,-0.062110,0.074123,0.001389,0.595660,0.171618,0.138461,-0.502776,0.737497,0


In [50]:
# Dataset dummied and scaled in other notebook
df = pd.read_csv('./osi_scaled_clean.csv')

In [51]:
df.Revenue = df.Revenue.apply(lambda x: 1 if x else 0)

In [52]:
df.head(20)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_Other,VisitorType_Returning_Visitor,Revenue
0,0.000000,0.000000,0.0,0.0,0.001418,0.000000,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0.000000,0.000000,0.0,0.0,0.002837,0.001000,0.000000,0.500000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0.000000,0.000000,0.0,0.0,0.001418,0.000000,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,0.000000,0.000000,0.0,0.0,0.002837,0.000042,0.250000,0.700000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0.000000,0.000000,0.0,0.0,0.014184,0.009809,0.100000,0.250000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
5,0.000000,0.000000,0.0,0.0,0.026950,0.002411,0.078947,0.122807,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
6,0.000000,0.000000,0.0,0.0,0.001418,0.000000,1.000000,1.000000,0.0,0.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
7,0.037037,0.000000,0.0,0.0,0.000000,0.000000,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
8,0.000000,0.000000,0.0,0.0,0.002837,0.000578,0.000000,0.500000,0.0,0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
9,0.000000,0.000000,0.0,0.0,0.004255,0.011536,0.000000,0.111111,0.0,0.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [53]:
# spaces.box??
# spaces.Discrete??

In [67]:
Reward_adjustment = 2
# Refers to Reward structure. Is imbalance Ratio of classes.
# This ratio/reward maybe tuned to improve model Recall
lamb_da = (1908/10422) * Reward_adjustment

In [68]:
lamb_da

0.36614853195164077

In [69]:
class DataFrameEnv(gym.Env):
 
    def __init__(self):
        
        #Init Action Space with to Discrete steps. 0 or 1, Left or right, On or off.
        self.action_space = spaces.Discrete(2)
        #Init Observation space 26x1 that contain continous values. change shape to fit any dataframe(in theory)
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(26,), dtype=np.float32)
        
        #Init the rest of DataFrameEnv class variables
        self.episode_df = None
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None
        self.episode = None
        self.test_set_no_labels = None
        self.test_labels = None

    def reset(self):
        # Reset function randomly samples Dataframe, Splits off labels and converts to list
        self.episode_df = df.sample(frac=0.8)
        self.episode = self.episode_df.drop(columns=['Revenue']).values.tolist()
        self.true_labels = self.episode_df['Revenue'].values.tolist()
        
        try:
            # Remove one row from sampled DF 
            self.state = self.episode.pop()
        except:
            self.state = None

        self.steps_beyond_done = None
        # Return to agent as a state.
        return np.array(self.state)

    
    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        
        # Pull off next row and assign to state to be returned to agent from this function
        state = self.episode.pop()
        

        done = False
        true_label = self.true_labels.pop()
        # Compare agents prediction with label and assign reward
        
        if true_label is not None:
            
            if (true_label == 1) & (action == true_label):
                reward = 1
            
            elif (true_label == 0) & (action == true_label):
                reward = lamb_da
            
            elif (true_label == 0) & (action != true_label):
                reward = -lamb_da
            
            else:
                reward = -1
                done = True
        else:
            reward = 0
            done = True
        # Return Next state, reward for action in current state and if done/ stop episode.
        return np.array(state), reward, done, {}

    def validate(self):
        # Validation function compare predictions to actuals on test set to build Confusion Matrix
        test_set = df[~df.index.isin(self.episode_df.index)]
        self.test_set_no_labels = test_set.drop(columns=['Revenue'])
        self.test_labels = test_set['Revenue']
        
        return self.test_set_no_labels, self.test_labels

In [70]:
# Init Environment
env = DataFrameEnv()

In [71]:
# Hyperparameters
H = 52 # number of hidden layer neurons
batch_size = 6 # how many episodes before doing a policy update
learning_rate = 0.009 # How big/small the steps are in the gradient ascent
gamma = 0.99 # discount factor for reward
learning_decay = 50 #Learning rate decay (not used currently)
D = 26 # input dimensionality
total_episodes = 1000 #duhh

In [72]:
def discount_rewards(r):
    # Calculates a reward to reduce rewards gained towards the end of an episode.
    # Not used except for gamma which is used to regulate prioritization of current vs future rewards.
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [73]:
tf.compat.v1.reset_default_graph()

#This defines the network as it goes from taking an observation of the environment to 
#giving a probability of chosing to the action 0 or 1.
observations = tf.compat.v1.placeholder(tf.float32, [None,D] , name="input_x")
W1 = tf.compat.v1.get_variable("W1", shape=[D, H],
           initializer=tf.initializers.GlorotUniform())
# first layer activation function was Tanh for PCA inputs to capture negative value information 
# changed to relu for modelling on original scaled dataset
layer1 = tf.nn.relu(tf.matmul(observations,W1))
W11 = tf.compat.v1.get_variable("W11", shape=[H, 8],
           initializer=tf.initializers.GlorotUniform())


layer11 = tf.nn.relu(tf.matmul(layer1, W11))
W2 = tf.compat.v1.get_variable("W2", shape=[8, 1],
           initializer=tf.initializers.GlorotUniform())

score = tf.matmul(layer11,W2)

probability = tf.nn.sigmoid(score)

#From here we define the parts of the network needed for learning a good policy.
tvars = tf.compat.v1.trainable_variables()
input_y = tf.compat.v1.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.compat.v1.placeholder(tf.float32,name="reward_signal")

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
loglik = tf.compat.v1.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages)

newGrads = tf.gradients(loss,tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradeients after every episode in order to account for noise in the reward signal.
adam = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
W1Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad1")
W11Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad11")# Placeholders to send the final gradients through when we update.
W2Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W11Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))

In [74]:
#stores states, actions, rewards for an episode
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
episode_rewards = []

#lists that maybe used to visualise model training/updates
policy_gradient = []
Q_approx = []
loss_list = []
predictions = []

In [75]:
init = tf.compat.v1.global_variables_initializer()
# Launch the graph
with tf.compat.v1.Session() as sess:
    
    sess.run(init)
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # Make sure the observation is in a shape the network can handle.
        x = np.reshape(observation,[1,D])
        
        # Run the policy network and get an action to take. 
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        
        xs.append(x) # observation
        y = 1 if action == 0 else 0 # a "fake label"
        ys.append(y)

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        
        reward_sum += reward

        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)
        
        
        if done: 
            episode_number += 1
            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            episode_rewards.append(np.sum(drs))
            tfp = tfps
            xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[] # reset array memory

            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            
            
            # Get the gradient for this episode, and save it in the gradBuffer
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes, then update the policy network with our gradients.
            if episode_number % batch_size == 0: 
                policy_gradient.append(gradBuffer)
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W11Grad:gradBuffer[1],W2Grad:gradBuffer[2]})

                loss_list.append(loss)
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                
                print('Average reward for episode %f.  Total average reward %f.' % (reward_sum//batch_size, running_reward//batch_size))
                
                reward_sum = 0

            observation = env.reset()
    #get test set with labels. 
    test_set, test_labels = env.validate()
    
    for i in test_set.values:
        # predict on each test row and append to list
        prediction = sess.run(probability, feed_dict={observations:np.reshape(i,[1,D])})
        predictions.append(prediction)
        
            
print(episode_number,'Episodes completed.')


Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 1.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode -2.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward 

In [76]:
predictions

[array([[1.1225107e-08]], dtype=float32),
 array([[1.2371716e-07]], dtype=float32),
 array([[1.105345e-05]], dtype=float32),
 array([[0.00216172]], dtype=float32),
 array([[0.0194863]], dtype=float32),
 array([[1.7546357e-08]], dtype=float32),
 array([[0.0005518]], dtype=float32),
 array([[6.919623e-05]], dtype=float32),
 array([[0.00010195]], dtype=float32),
 array([[0.00031363]], dtype=float32),
 array([[0.18191291]], dtype=float32),
 array([[0.00077057]], dtype=float32),
 array([[0.66486454]], dtype=float32),
 array([[0.00030208]], dtype=float32),
 array([[0.7799046]], dtype=float32),
 array([[0.02699403]], dtype=float32),
 array([[1.8506368e-08]], dtype=float32),
 array([[0.00083138]], dtype=float32),
 array([[0.01095333]], dtype=float32),
 array([[0.1816055]], dtype=float32),
 array([[0.00035664]], dtype=float32),
 array([[0.00040757]], dtype=float32),
 array([[0.02270149]], dtype=float32),
 array([[0.0005222]], dtype=float32),
 array([[0.13048838]], dtype=float32),
 array([[0.000